# 安裝和配置ngrok

In [ ]:
!pip install pyngrok
!ngrok authtoken 2mMwXocarPN8zicqdaHr26fxQYA_njrcukttMMMfbvciSs5X #!ngrok authtoken <YOUR_NGROK_AUTH_TOKEN>
# 確保你輸入了你的ngrok token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install flask-ngrok

In [ ]:
get_ipython().system('pip install scikit-fuzzy')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 13.9 MB/s eta 0:00:00


# 刪除DB

In [ ]:
rm /content/recognition_history.db

# flask執行評分

In [ ]:
import os
import cv2
import base64
import sqlite3
import uuid  # 用於生成唯一的圖片名稱
import json   # 用於處理 JSON 資料
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from flask import Flask, request, send_from_directory, redirect, url_for, session, abort
from werkzeug.utils import secure_filename
from pyngrok import ngrok

# YOLO 模型配置和權重文件路徑
yolo_weights = '/content/darknet/backup/my_yolov4-tiny-custom_final.weights'
yolo_cfg = '/content/darknet/cfg/my_yolov4-tiny-custom.cfg'
yolo_names = '/content/darknet/data/obj.names'

# 讀取類別名稱
with open(yolo_names, "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Flask 應用設置
app = Flask(__name__)
app.secret_key = 'your_secret_key_here'  # 設置密鑰以使用 session

UPLOAD_FOLDER = '/content/uploads'
RESULT_FOLDER = '/content/results'
TEMP_FOLDER = '/content/temp'  # 用於存儲臨時數據
DATABASE_PATH = '/content/recognition_history.db'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['RESULT_FOLDER'] = RESULT_FOLDER
app.config['TEMP_FOLDER'] = TEMP_FOLDER

# 創建上傳、結果和臨時資料夾
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(RESULT_FOLDER, exist_ok=True)
os.makedirs(TEMP_FOLDER, exist_ok=True)

# 允許的文件類型
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# 初始化 SQLite 資料庫
def init_db():
    conn = sqlite3.connect(DATABASE_PATH)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            image_base64 TEXT,
            result_text TEXT,
            pigeon_quality TEXT,
            pigeon_wing REAL,
            pigeon_eye REAL,
            pigeon_eye_serrated REAL,
            race_pigeon_score REAL,  -- 新增欄位
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    ''')

    # 嘗試新增 race_pigeon_score 欄位，如果已存在則忽略
    try:
        cursor.execute('ALTER TABLE history ADD COLUMN race_pigeon_score REAL')
    except sqlite3.OperationalError:
        pass  # 欄位已存在

    conn.commit()
    conn.close()

# 將圖片轉換為 base64 格式
def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# 記錄 YOLO 檢測結果到資料庫
def log_recognition(image_base64_list, result_text_list, pigeon_quality, pigeon_wing, pigeon_eye, pigeon_eye_serrated, race_pigeon_score):
    conn = sqlite3.connect(DATABASE_PATH)
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO history (image_base64, result_text, pigeon_quality, pigeon_wing, pigeon_eye, pigeon_eye_serrated, race_pigeon_score)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (
        json.dumps(image_base64_list),
        json.dumps(result_text_list),
        pigeon_quality,
        pigeon_wing,
        pigeon_eye,
        pigeon_eye_serrated,
        race_pigeon_score
    ))
    conn.commit()
    # 返回插入的 id
    last_id = cursor.lastrowid
    conn.close()
    return last_id

# 定義模糊變數和隸屬函數
# Input variables
pigeon_wing_var = ctrl.Antecedent(np.arange(0.0, 1.1, 0.1), 'pigeon_wing')  # 更新範圍至 0.0 - 1.0
pigeon_eye_var = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'pigeon_eye')
pigeon_eye_serrated_var = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'pigeon_eye_serrated')

# Output variable
race_pigeon_var = ctrl.Consequent(np.arange(0, 101, 1), 'race_pigeon')

# Define membership functions for pigeon_wing
pigeon_wing_var['Low'] = fuzz.trimf(pigeon_wing_var.universe, [0.0, 0.6, 0.8])
pigeon_wing_var['Medium'] = fuzz.trimf(pigeon_wing_var.universe, [0.7, 0.8, 0.9])
pigeon_wing_var['High'] = fuzz.trapmf(pigeon_wing_var.universe, [0.85, 0.95, 1.0, 1.0])

# Define membership functions for pigeon_eye
pigeon_eye_var['Low'] = fuzz.trapmf(pigeon_eye_var.universe, [0, 0, 0.25, 0.5])
pigeon_eye_var['Medium'] = fuzz.trimf(pigeon_eye_var.universe, [0.25, 0.5, 0.75])
pigeon_eye_var['High'] = fuzz.trapmf(pigeon_eye_var.universe, [0.5, 0.75, 1.0, 1.0])

# Define membership functions for pigeon_eye_serrated
pigeon_eye_serrated_var['Low'] = fuzz.trimf(pigeon_eye_serrated_var.universe, [0.0, 0.0, 0.5])
pigeon_eye_serrated_var['Medium'] = fuzz.trapmf(pigeon_eye_serrated_var.universe, [0.5, 0.6, 0.7, 0.85])
pigeon_eye_serrated_var['High'] = fuzz.trapmf(pigeon_eye_serrated_var.universe, [0.7, 0.85, 1.0, 1.0])

# Define membership functions for race_pigeon
race_pigeon_var['Low'] = fuzz.trapmf(race_pigeon_var.universe, [0, 0, 50, 60])
race_pigeon_var['Medium'] = fuzz.trimf(race_pigeon_var.universe, [50, 65, 80])
race_pigeon_var['High'] = fuzz.trapmf(race_pigeon_var.universe, [75, 85, 100, 100])




# 定義模糊規則
rule1 = ctrl.Rule(
    pigeon_wing_var['High'] &
    pigeon_eye_var['High'] &
    pigeon_eye_serrated_var['High'],
    race_pigeon_var['High']
)

rule2 = ctrl.Rule(
    pigeon_wing_var['High'] &
    pigeon_eye_var['High'] &
    pigeon_eye_serrated_var['Medium'],
    race_pigeon_var['High']
)

rule3 = ctrl.Rule(
    pigeon_wing_var['Medium'] &
    pigeon_eye_var['Medium'] &
    pigeon_eye_serrated_var['Medium'],
    race_pigeon_var['Medium']
)

rule4 = ctrl.Rule(
    pigeon_wing_var['Low'] &
    pigeon_eye_var['Medium'] &
    pigeon_eye_serrated_var['Medium'],
    race_pigeon_var['Medium']
)

rule5 = ctrl.Rule(
    pigeon_wing_var['Low'] &
    pigeon_eye_var['Low'] &
    pigeon_eye_serrated_var['Low'],
    race_pigeon_var['Low']
)

rule6 = ctrl.Rule(
    pigeon_wing_var['Medium'] &
    pigeon_eye_var['High'] &
    pigeon_eye_serrated_var['Low'],
    race_pigeon_var['Medium']
)

rule7 = ctrl.Rule(
    pigeon_wing_var['High'] &
    pigeon_eye_var['Medium'] &
    pigeon_eye_serrated_var['High'],
    race_pigeon_var['High']
)

rule9 = ctrl.Rule(
    pigeon_wing_var['Low'] &
    (pigeon_eye_var['Low'] | pigeon_eye_var['Medium']) &
    (pigeon_eye_serrated_var['Low'] | pigeon_eye_serrated_var['Medium']),
    race_pigeon_var['Low']
)

# 擴展模糊規則以覆蓋更多輸入組合
rule10 = ctrl.Rule(
    pigeon_wing_var['Low'] &
    pigeon_eye_var['High'] &
    pigeon_eye_serrated_var['High'],
    race_pigeon_var['Medium']
)

rule11 = ctrl.Rule(
    pigeon_wing_var['Low'] &
    pigeon_eye_var['High'] &
    pigeon_eye_serrated_var['Medium'],
    race_pigeon_var['Medium']
)

rule12 = ctrl.Rule(
    pigeon_wing_var['Low'] &
    pigeon_eye_var['High'] &
    pigeon_eye_serrated_var['High'],
    race_pigeon_var['Medium']
)

rule13 = ctrl.Rule(
    pigeon_wing_var['Low'] &
    pigeon_eye_var['High'] &
    pigeon_eye_serrated_var['Medium'],
    race_pigeon_var['Medium']
)

rule14 = ctrl.Rule(
    pigeon_wing_var['Medium'] &
    pigeon_eye_var['Low'] &
    pigeon_eye_serrated_var['High'],
    race_pigeon_var['Medium']
)

# 修改 rule15 和 rule16，加入 pigeon_wing_var 的條件，確保不包括 Low2
rule15 = ctrl.Rule(
    pigeon_eye_var['High'] & (pigeon_wing_var['Medium'] | pigeon_wing_var['High']),
    race_pigeon_var['High']
)

rule16 = ctrl.Rule(
    pigeon_eye_serrated_var['High'] & (pigeon_wing_var['Medium'] | pigeon_wing_var['High']),
    race_pigeon_var['High']
)

# 添加預設規則以覆蓋所有其他情況
rule17 = ctrl.Rule(
    (pigeon_wing_var['Low'] | pigeon_wing_var['Medium'] | pigeon_wing_var['High']) &
    (pigeon_eye_var['Low'] | pigeon_eye_var['Medium'] | pigeon_eye_var['High']) &
    (pigeon_eye_serrated_var['Low'] | pigeon_eye_serrated_var['Medium'] | pigeon_eye_serrated_var['High']),
    race_pigeon_var['Medium']
)

# 建立控制系統
race_pigeon_ctrl = ctrl.ControlSystem([
    rule1, rule2, rule3, rule4, rule5, rule6, rule7,
    rule9, rule10, rule11, rule12, rule13, rule14,
    rule15, rule16, rule17
])


# 使用 YOLO 進行物體檢測並返回檢測到的特徵值和標註後的圖片
def detect_objects(image_path):
    net = cv2.dnn.readNet(yolo_weights, yolo_cfg)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("無法讀取圖片，請檢查圖片路徑。")
    height, width, _ = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    inner_wing_height = 0
    outer_wing_height = 0
    result_text = []

    # 變數初始化
    hidden_eye_confidence = 0
    pigeon_eye_confidence = 0
    serrated_present = False

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = f"{classes[class_ids[i]]}: {confidences[i]:.2f} - Height: {h}"
            result_text.append(label)

            if classes[class_ids[i]] == "inner wing":
                inner_wing_height += h  # 累加高度
            elif classes[class_ids[i]] == "outer wing":
                outer_wing_height += h
            elif classes[class_ids[i]] == "hidden eye":
                hidden_eye_confidence += 0.5 * confidences[i]
            elif classes[class_ids[i]] == "dominant eye":
                pigeon_eye_confidence = max(pigeon_eye_confidence, 1.0 * confidences[i], 0.5)
            elif classes[class_ids[i]] == "serrated":
                serrated_present = True

            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 計算 pigeon_wing
    if inner_wing_height == 0 or outer_wing_height == 0:
        pigeon_wing = None  # 缺失
    else:
        # 修改計算規則為 min / max，確保結果在 0 到 1 之間
        pigeon_wing = min(inner_wing_height, outer_wing_height) / max(inner_wing_height, outer_wing_height)

    # 計算 pigeon_eye
    if pigeon_eye_confidence >= 0.5:
        pigeon_eye = pigeon_eye_confidence
    elif hidden_eye_confidence > 0:
        pigeon_eye = hidden_eye_confidence
    else:
        pigeon_eye = None  # 缺失

    # 計算 pigeon_eye_serrated
    if serrated_present:
        pigeon_eye_serrated = 1.0
    else:
        pigeon_eye_serrated = 0.8  # 預設值

    return {
        'result_text': result_text,
        'pigeon_wing': pigeon_wing,
        'pigeon_eye': pigeon_eye,
        'pigeon_eye_serrated': pigeon_eye_serrated,
        'annotated_image': img
    }

# 主頁面
@app.route('/')
def index():
    return '''
    <!doctype html>
    <html lang="zh-TW">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>上傳圖片進行物體檢測</title>
        <style>
            body { background-color: #f8f9fa; font-family: Arial, sans-serif; }
            h1 { text-align: center; color: #333; }
            form { display: flex; flex-direction: column; align-items: center; margin-top: 20px; }
            input[type="file"], button { margin: 10px; padding: 10px; border-radius: 5px; border: 1px solid #ccc; }
            button { background-color: #007bff; color: white; cursor: pointer; }
            button:hover { background-color: #0056b3; }
        </style>
    </head>
    <body>
        <h1>上傳圖片進行物體檢測</h1>
        <form method="post" enctype="multipart/form-data" action="/upload">
            <input type="file" name="files" class="form-control" multiple required>
            <button type="submit" class="btn btn-primary">上傳並分析</button>
        </form>
        <br>
        <a href="/history">查看歷史紀錄</a>
    </body>
    </html>
    '''

# 路由來服務結果圖片
@app.route('/results/<filename>')
def serve_result_image(filename):
    return send_from_directory(app.config['RESULT_FOLDER'], filename)

# 處理圖片上傳並進行物體檢測
@app.route('/upload', methods=['POST'])
def upload_file_route():
    if 'files' not in request.files:
        return '沒有上傳文件'

    files = request.files.getlist('files')

    if not files or any(file.filename == '' for file in files):
        return '沒有選擇文件'

    image_base64_list = []
    annotated_filenames = []
    result_text_list = []
    pigeon_wing_values = []
    pigeon_eye_values = []
    pigeon_eye_serrated_values = []
    missing_features = {'pigeon_wing': False, 'pigeon_eye': False}

    for file in files:
        if file and allowed_file(file.filename):
            original_filename = secure_filename(file.filename)
            unique_filename = f"{uuid.uuid4().hex}_{original_filename}"
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], unique_filename)
            file.save(filepath)

            try:
                result = detect_objects(filepath)
                annotated_image = result['annotated_image']

                # 生成唯一的結果圖片名稱
                annotated_filename = f"annotated_{unique_filename}"
                result_image_path = os.path.join(app.config['RESULT_FOLDER'], annotated_filename)
                cv2.imwrite(result_image_path, annotated_image)

                # 將圖片轉換為 base64
                image_base64 = image_to_base64(result_image_path)
                image_base64_list.append(image_base64)
                annotated_filenames.append(annotated_filename)

                # 收集檢測結果文字（將不在網頁上顯示，但保留在資料庫中）
                result_text_list.extend(result['result_text'])

                # 收集特徵值
                if result['pigeon_wing'] is not None:
                    pigeon_wing_values.append(result['pigeon_wing'])
                else:
                    missing_features['pigeon_wing'] = True

                if result['pigeon_eye'] is not None:
                    pigeon_eye_values.append(result['pigeon_eye'])
                else:
                    missing_features['pigeon_eye'] = True

                pigeon_eye_serrated_values.append(result['pigeon_eye_serrated'])

            except Exception as e:
                return f'處理圖片 "{original_filename}" 時發生錯誤: {str(e)}'

    # 判斷是否所有圖片都缺失某個特徵
    all_pigeon_wing_missing = len(pigeon_wing_values) == 0
    all_pigeon_eye_missing = len(pigeon_eye_values) == 0

    missing_features = {}
    if all_pigeon_wing_missing:
        missing_features['pigeon_wing'] = True
    if all_pigeon_eye_missing:
        missing_features['pigeon_eye'] = True

    if missing_features:
        # 生成一個唯一的ID來存儲臨時數據
        temp_id = uuid.uuid4().hex
        temp_data = {
            'image_base64_list': image_base64_list,
            'annotated_filenames': annotated_filenames,
            'result_text_list': result_text_list,
            'pigeon_wing_values': pigeon_wing_values,        # 添加
            'pigeon_eye_values': pigeon_eye_values,          # 添加
            'pigeon_eye_serrated_values': pigeon_eye_serrated_values,
            'missing_features': missing_features
        }
        temp_filepath = os.path.join(app.config['TEMP_FOLDER'], f"{temp_id}.json")
        with open(temp_filepath, 'w') as f:
            json.dump(temp_data, f)

        # 將temp_id存儲在session中
        session['temp_id'] = temp_id

        return redirect(url_for('input_missing'))

    # 如果所有特徵都檢測到，進行分析
    # 匯總特徵值
    avg_pigeon_wing = np.mean(pigeon_wing_values) if pigeon_wing_values else 1.0
    avg_pigeon_eye = np.mean(pigeon_eye_values) if pigeon_eye_values else 1.0
    max_pigeon_eye_serrated = np.max(pigeon_eye_serrated_values) if pigeon_eye_serrated_values else 0.8

    # 創建新的控制系統模擬實例
    sim = ctrl.ControlSystemSimulation(race_pigeon_ctrl)

    # 傳遞匯總後的輸入到控制系統
    sim.input['pigeon_wing'] = avg_pigeon_wing
    sim.input['pigeon_eye'] = avg_pigeon_eye
    sim.input['pigeon_eye_serrated'] = max_pigeon_eye_serrated

    # 調試輸入值
    print(f"Inputs: pigeon_wing={avg_pigeon_wing}, pigeon_eye={avg_pigeon_eye}, pigeon_eye_serrated={max_pigeon_eye_serrated}")

    # 計算結果
    sim.compute()

    # 調試輸出值
    print(f"Simulation output: {sim.output}")

    race_pigeon_score = sim.output.get('race_pigeon')

    if race_pigeon_score is None:
        return '無法計算賽鴿評分，請檢查輸入值。'

    # 根據 race_pigeon_score 分類
    if race_pigeon_score <= 60:
        pigeon_quality = "這隻賽鴿不好。"
    elif 60 < race_pigeon_score <= 78:
        pigeon_quality = "這隻賽鴿中等。"
    else:
        pigeon_quality = "這隻賽鴿好！"

    # 記錄到資料庫
    record_id = log_recognition(
        image_base64_list,
        result_text_list,
        pigeon_quality,
        avg_pigeon_wing,
        avg_pigeon_eye,
        max_pigeon_eye_serrated,
        race_pigeon_score  # 傳遞分數
    )

    # 構建結果頁面的 HTML（移除檢測到的物體文字）
    result_html = f'''
    <!doctype html>
    <html lang="zh-TW">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>分析結果</title>
        <style>
            body {{ background-color: #f8f9fa; font-family: Arial, sans-serif; }}
            h1 {{ text-align: center; color: #333; }}
            .result-container {{ display: flex; flex-wrap: wrap; justify-content: center; }}
            .result {{ border: 1px solid #ddd; border-radius: 5px; padding: 20px; margin: 10px; width: 300px; background-color: #fff; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }}
            img {{ display: block; margin: 0 auto 10px; max-width: 100%; height: auto; }}
            .score {{ text-align: center; font-size: 1.2em; color: #555; }}
            .metrics {{ text-align: center; margin-top: 10px; }}
            .metrics table {{ margin: 0 auto; border-collapse: collapse; }}
            .metrics th, .metrics td {{ padding: 8px 12px; border: 1px solid #ddd; }}
            a {{ display: block; text-align: center; margin-top: 10px; text-decoration: none; background-color: #007bff; color: white; padding: 8px 16px; border-radius: 5px; }}
            a:hover {{ background-color: #0056b3; }}
        </style>
    </head>
    <body>
        <h1>分析結果</h1>
        <div class="result-container">
    '''

    # 顯示所有標註後的圖片（不顯示檢測到的物體文字）
    for annotated_filename in annotated_filenames:
        result_html += f'''
            <div class="result">
                <img src="/results/{annotated_filename}" alt="檢測結果圖片">
            </div>
        '''

    # 顯示一次性的評分結果和詳細指標
    result_html += f'''
            <div class="result">
                <h2>賽鴿評估結果：{pigeon_quality}</h2>
                <div class="score">模糊評分 (Race Pigeon Score): {race_pigeon_score:.2f}</div>
                <div class="metrics">
                    <h3>詳細指標</h3>
                    <table>
                        <tr>
                            <th>指標</th>
                            <th>值</th>
                        </tr>
                        <tr>
                            <td>pigeon_wing</td>
                            <td>{avg_pigeon_wing:.2f}</td>
                        </tr>
                        <tr>
                            <td>pigeon_eye</td>
                            <td>{avg_pigeon_eye:.2f}</td>
                        </tr>
                        <tr>
                            <td>pigeon_eye_serrated</td>
                            <td>{max_pigeon_eye_serrated:.2f}</td>
                        </tr>
                    </table>
                </div>
                <a href="/share/{record_id}">分享此結果</a>
            </div>
    '''

    result_html += '''
        </div>
        <br><a href="/">再次上傳圖片</a>
    </body>
    </html>
    '''

    return result_html

# 新增路由來處理缺失特徵的輸入
@app.route('/input_missing', methods=['GET', 'POST'])
def input_missing():
    temp_id = session.get('temp_id')
    if not temp_id:
        return redirect(url_for('index'))

    temp_filepath = os.path.join(app.config['TEMP_FOLDER'], f"{temp_id}.json")
    if not os.path.exists(temp_filepath):
        return '臨時數據不存在或已過期。'

    if request.method == 'GET':
        with open(temp_filepath, 'r') as f:
            temp_data = json.load(f)

        missing_features = temp_data.get('missing_features', {})
        if not missing_features:
            return redirect(url_for('index'))

        form_html = '''
        <!doctype html>
        <html lang="zh-TW">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>輸入缺失的特徵值</title>
            <style>
                body { background-color: #f8f9fa; font-family: Arial, sans-serif; }
                h1 { text-align: center; color: #333; }
                form { display: flex; flex-direction: column; align-items: center; margin-top: 20px; }
                input[type="number"], button { margin: 10px; padding: 10px; border-radius: 5px; border: 1px solid #ccc; width: 200px; }
                button { background-color: #28a745; color: white; cursor: pointer; }
                button:hover { background-color: #218838; }
            </style>
        </head>
        <body>
            <h1>輸入缺失的特徵值</h1>
            <form method="post" action="/input_missing">
        '''

        # 根據缺失的特徵動態生成輸入欄位
        if missing_features.get('pigeon_wing'):
            form_html += '''
                <label for="pigeon_wing">Pigeon Wing (0 - 1.0):</label>
                <input type="number" id="pigeon_wing" name="pigeon_wing" step="0.01" min="0" max="1.0" required>
            '''
        if missing_features.get('pigeon_eye'):
            form_html += '''
                <label for="pigeon_eye">Pigeon Eye (0 - 1.0):</label>
                <input type="number" id="pigeon_eye" name="pigeon_eye" step="0.01" min="0" max="1.0" required>
            '''

        form_html += '''
                <button type="submit">提交並分析</button>
            </form>
            <br>
            <a href="/">返回首頁</a>
        </body>
        </html>
        '''

        return form_html

    elif request.method == 'POST':
        with open(temp_filepath, 'r') as f:
            temp_data = json.load(f)

        missing_features = temp_data.get('missing_features', {})
        user_inputs = {}
        errors = []

        # 驗證並獲取用戶輸入的缺失特徵值
        if missing_features.get('pigeon_wing'):
            try:
                pigeon_wing = float(request.form['pigeon_wing'])
                if not (0.0 <= pigeon_wing <= 1.0):
                    raise ValueError
                user_inputs['pigeon_wing'] = pigeon_wing
            except:
                errors.append('pigeon_wing 的值無效，請返回重新輸入。')

        if missing_features.get('pigeon_eye'):
            try:
                pigeon_eye = float(request.form['pigeon_eye'])
                if not (0.0 <= pigeon_eye <= 1.0):
                    raise ValueError
                user_inputs['pigeon_eye'] = pigeon_eye
            except:
                errors.append('pigeon_eye 的值無效，請返回重新輸入。')

        if errors:
            return '<br>'.join(errors)

        # 從臨時數據中獲取之前的檢測結果
        image_base64_list = temp_data.get('image_base64_list', [])
        annotated_filenames = temp_data.get('annotated_filenames', [])
        result_text_list = temp_data.get('result_text_list', [])
        pigeon_eye_serrated_values = temp_data.get('pigeon_eye_serrated_values', [])
        pigeon_wing_values = temp_data.get('pigeon_wing_values', [])
        pigeon_eye_values = temp_data.get('pigeon_eye_values', [])

        # 收集特徵值
        if 'pigeon_wing' in user_inputs:
            avg_pigeon_wing = user_inputs['pigeon_wing']
        else:
            avg_pigeon_wing = np.mean([v for v in pigeon_wing_values if v is not None]) if pigeon_wing_values else 0.5  # 使用中間值

        if 'pigeon_eye' in user_inputs:
            avg_pigeon_eye = user_inputs['pigeon_eye']
        else:
            avg_pigeon_eye = np.mean([v for v in pigeon_eye_values if v is not None]) if pigeon_eye_values else 0.5  # 使用中間值

        # 計算 pigeon_eye_serrated 的最大值
        max_pigeon_eye_serrated = np.max(pigeon_eye_serrated_values) if pigeon_eye_serrated_values else 0.8

        # 創建新的控制系統模擬實例
        sim = ctrl.ControlSystemSimulation(race_pigeon_ctrl)

        # 傳遞匯總後的輸入到控制系統
        sim.input['pigeon_wing'] = avg_pigeon_wing
        sim.input['pigeon_eye'] = avg_pigeon_eye
        sim.input['pigeon_eye_serrated'] = max_pigeon_eye_serrated

        # 調試輸入值
        print(f"Inputs: pigeon_wing={avg_pigeon_wing}, pigeon_eye={avg_pigeon_eye}, pigeon_eye_serrated={max_pigeon_eye_serrated}")

        # 計算結果
        sim.compute()

        # 調試輸出值
        print(f"Simulation output: {sim.output}")

        race_pigeon_score = sim.output.get('race_pigeon')

        if race_pigeon_score is None:
            return '無法計算賽鴿評分，請檢查輸入值。'

        # 根據 race_pigeon_score 分類
        if race_pigeon_score <= 60:
            pigeon_quality = "這隻賽鴿不好。"
        elif 60 < race_pigeon_score <= 78:
            pigeon_quality = "這隻賽鴿中等。"
        else:
            pigeon_quality = "這隻賽鴿好！"

        # 記錄到資料庫
        record_id = log_recognition(
            image_base64_list,
            result_text_list,
            pigeon_quality,
            avg_pigeon_wing,
            avg_pigeon_eye,
            max_pigeon_eye_serrated,
            race_pigeon_score  # 傳遞分數
        )

        # 清除臨時數據和session中的temp_id
        os.remove(temp_filepath)
        session.pop('temp_id', None)

        # 構建結果頁面的 HTML（移除檢測到的物體文字）
        result_html = f'''
        <!doctype html>
        <html lang="zh-TW">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>分析結果</title>
            <style>
                body {{ background-color: #f8f9fa; font-family: Arial, sans-serif; }}
                h1 {{ text-align: center; color: #333; }}
                .result-container {{ display: flex; flex-wrap: wrap; justify-content: center; }}
                .result {{ border: 1px solid #ddd; border-radius: 5px; padding: 20px; margin: 10px; width: 300px; background-color: #fff; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }}
                img {{ display: block; margin: 0 auto 10px; max-width: 100%; height: auto; }}
                .score {{ text-align: center; font-size: 1.2em; color: #555; }}
                .metrics {{ text-align: center; margin-top: 10px; }}
                .metrics table {{ margin: 0 auto; border-collapse: collapse; }}
                .metrics th, .metrics td {{ padding: 8px 12px; border: 1px solid #ddd; }}
                a {{ display: block; text-align: center; margin-top: 10px; text-decoration: none; background-color: #007bff; color: white; padding: 8px 16px; border-radius: 5px; }}
                a:hover {{ background-color: #0056b3; }}
            </style>
        </head>
        <body>
            <h1>分析結果</h1>
            <div class="result-container">
        '''

        # 顯示所有標註後的圖片（不顯示檢測到的物體文字）
        for annotated_filename in annotated_filenames:
            result_html += f'''
                <div class="result">
                    <img src="/results/{annotated_filename}" alt="檢測結果圖片">
                </div>
            '''

        # 顯示一次性的評分結果和詳細指標
        result_html += f'''
                <div class="result">
                    <h2>賽鴿評估結果：{pigeon_quality}</h2>
                    <div class="score">模糊評分 (Race Pigeon Score): {race_pigeon_score:.2f}</div>
                    <div class="metrics">
                        <h3>詳細指標</h3>
                        <table>
                            <tr>
                                <th>指標</th>
                                <th>值</th>
                            </tr>
                            <tr>
                                <td>pigeon_wing</td>
                                <td>{avg_pigeon_wing:.2f}</td>
                            </tr>
                            <tr>
                                <td>pigeon_eye</td>
                                <td>{avg_pigeon_eye:.2f}</td>
                            </tr>
                            <tr>
                                <td>pigeon_eye_serrated</td>
                                <td>{max_pigeon_eye_serrated:.2f}</td>
                            </tr>
                        </table>
                    </div>
                    <a href="/share/{record_id}">分享此結果</a>
                </div>
        '''

        result_html += '''
            </div>
            <br><a href="/">再次上傳圖片</a>
        </body>
        </html>
        '''

        return result_html

# 顯示歷史紀錄
@app.route('/history')
def view_history():
    conn = sqlite3.connect(DATABASE_PATH)
    cursor = conn.cursor()
    # 修改SELECT語句以包含新增的欄位 race_pigeon_score
    cursor.execute('SELECT id, image_base64, pigeon_quality, pigeon_wing, pigeon_eye, pigeon_eye_serrated, race_pigeon_score, timestamp FROM history ORDER BY timestamp DESC')
    rows = cursor.fetchall()
    conn.close()

    history_html = '''
    <!doctype html>
    <html lang="zh-TW">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>歷史紀錄</title>
        <style>
            body { background-color: #f8f9fa; font-family: Arial, sans-serif; }
            h1 { text-align: center; color: #333; }
            table { width: 100%; border-collapse: collapse; }
            th, td { padding: 10px; border: 1px solid #ddd; text-align: center; }
            img { max-width: 150px; height: auto; }
            a { display: block; text-align: center; margin-top: 20px; text-decoration: none; background-color: #007bff; color: white; padding: 10px 20px; border-radius: 5px; }
            a:hover { background-color: #0056b3; }
        </style>
    </head>
    <body>
        <h1>歷史紀錄</h1>
        <table>
            <thead>
                <tr>
                    <th>ID</th>
                    <th>圖片</th>
                    <th>賽鴿評估</th>
                    <th>指標</th>
                    <th>分數</th>
                    <th>時間</th>
                    <th>分享連結</th>
                </tr>
            </thead>
            <tbody>
    '''

    for row in rows:
        record_id = row[0]
        image_base64_list = json.loads(row[1])  # 解析 JSON 字串為列表
        pigeon_quality = row[2]
        pigeon_wing = row[3]
        pigeon_eye = row[4]
        pigeon_eye_serrated = row[5]
        race_pigeon_score = row[6]  # 新增
        timestamp = row[7]

        # 將多張圖片的 Base64 編碼組合起來顯示
        images_html = ''
        for image_base64 in image_base64_list:
            images_html += f'<img src="data:image/jpeg;base64,{image_base64}" alt="結果圖片"><br>'

        # 構建指標顯示
        metrics_html = f'''
            <table>
                <tr>
                    <th>指標</th>
                    <th>值</th>
                </tr>
                <tr>
                    <td>pigeon_wing</td>
                    <td>{pigeon_wing:.2f}</td>
                </tr>
                <tr>
                    <td>pigeon_eye</td>
                    <td>{pigeon_eye:.2f}</td>
                </tr>
                <tr>
                    <td>pigeon_eye_serrated</td>
                    <td>{pigeon_eye_serrated:.2f}</td>
                </tr>
            </table>
        '''

        # 顯示 race_pigeon_score 作為分數
        score_html = f"{race_pigeon_score:.2f}"

        history_html += f'''
            <tr>
                <td>{record_id}</td>
                <td>{images_html}</td>
                <td>{pigeon_quality}</td>
                <td>{metrics_html}</td>
                <td>{score_html}</td>
                <td>{timestamp}</td>
                <td><a href="/share/{record_id}">分享</a></td>
            </tr>
        '''

    history_html += '''
            </tbody>
        </table>
        <br><a href="/">返回首頁</a>
    </body>
    </html>
    '''

    return history_html

# 分享結果的頁面
@app.route('/share/<int:id>')
def share_result(id):
    conn = sqlite3.connect(DATABASE_PATH)
    cursor = conn.cursor()
    cursor.execute('SELECT image_base64, pigeon_quality, pigeon_wing, pigeon_eye, pigeon_eye_serrated, race_pigeon_score, timestamp FROM history WHERE id = ?', (id,))
    row = cursor.fetchone()
    conn.close()

    if row:
        image_base64_list = json.loads(row[0])
        pigeon_quality = row[1]
        pigeon_wing = row[2]
        pigeon_eye = row[3]
        pigeon_eye_serrated = row[4]
        race_pigeon_score = row[5]  # 新增
        timestamp = row[6]

        # 將多張圖片的 Base64 編碼組合起來顯示
        images_html = ''
        for image_base64 in image_base64_list:
            images_html += f'<img src="data:image/jpeg;base64,{image_base64}" alt="結果圖片" class="img-fluid"><br>'

        # 構建指標顯示
        metrics_html = f'''
            <table>
                <tr>
                    <th>指標</th>
                    <th>值</th>
                </tr>
                <tr>
                    <td>pigeon_wing</td>
                    <td>{pigeon_wing:.2f}</td>
                </tr>
                <tr>
                    <td>pigeon_eye</td>
                    <td>{pigeon_eye:.2f}</td>
                </tr>
                <tr>
                    <td>pigeon_eye_serrated</td>
                    <td>{pigeon_eye_serrated:.2f}</td>
                </tr>
            </table>
        '''

        result_html = f'''
        <!doctype html>
        <html lang="zh-TW">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>分享結果</title>
            <style>
                body {{ background-color: #f8f9fa; font-family: Arial, sans-serif; }}
                h1 {{ text-align: center; color: #333; }}
                img {{ display: block; margin: 20px auto; max-width: 90%; height: auto; }}
                .score {{ text-align: center; font-size: 1.2em; color: #555; }}
                .metrics {{ text-align: center; margin-top: 10px; }}
                .metrics table {{ margin: 0 auto; border-collapse: collapse; }}
                .metrics th, .metrics td {{ padding: 8px 12px; border: 1px solid #ddd; }}
                a {{ display: block; text-align: center; margin-top: 20px; text-decoration: none; background-color: #007bff; color: white; padding: 10px 20px; border-radius: 5px; }}
                a:hover {{ background-color: #0056b3; }}
            </style>
        </head>
        <body>
            <h1>分享的檢測結果</h1>
            {images_html}
            <h2>賽鴿評估結果：{pigeon_quality}</h2>
            <div class="score">模糊評分 (Race Pigeon Score): {race_pigeon_score:.2f}</div>
            <div class="metrics">
                <h3>詳細指標</h3>
                {metrics_html}
            </div>
            <p>檢測時間：{timestamp}</p>
            <br><a href="/">返回首頁</a>
        </body>
        </html>
        '''

        return result_html
    else:
        return '沒有找到該結果'

# 啟動應用
if __name__ == '__main__':
    init_db()
    # 啟動 ngrok 隧道
    public_url = ngrok.connect(5000)
    print("ngrok URL:", public_url)
    # 啟動 Flask 應用
    app.run(host='0.0.0.0', port=5000)
